In this notebook we give a proof of concept of unitary compiling using TFQ. 

In [2]:
%load_ext autoreload
%autoreload 2

import sympy 
import numpy as np 
import pandas as pd 
import tensorflow as tf
from utilities.circuit_database import CirqTranslater
from utilities.templates import *
from utilities.variational import Minimizer
from utilities.misc import get_qubits_involved, reindex_symbol, shift_symbols_down
import matplotlib.pyplot as plt 
import tensorflow_quantum as tfq
import cirq
from utilities.compiling import *
from utilities.misc import *
from utilities.simplifier import Simplifier
from utilities.discrimination import *
from utilities.idinserter import IdInserter
from utilities.evaluator import Evaluator
from utilities.gate_killer import GateKiller

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
np.random.seed(0)
translator = CirqTranslater(3, untouchable_blocks = [1])
translator.env_qubits = [2]
translator.encoder_id = 0
translator.channel_id = 1
translator.decoder_id = 2
simplifier = Simplifier(translator)
etas = [0.01, 1.]
minimizer = Minimizer(translator, mode="discrimination", params=etas)
killer = GateKiller(translator, mode="discrimination", params = etas)
inserter = IdInserter(translator.n_qubits, untouchable_blocks=translator.channel_id)
args_evaluator = {"n_qubits":translator.n_qubits, "problem":"acd","params":etas}
evaluator = Evaluator(args=args_evaluator, lower_bound_cost=minimizer.lower_bound_cost, nrun=0)

In [25]:
block_0 = concatenate_dbs([z_layer_db(translator, block_id=0), x_layer_db(translator, block_id=0), z_layer_db(translator, block_id=0), x_layer_db(translator, block_id=0)])
blo = 1
block_1 = concatenate_dbs([z_layer_db(translator, block_id=blo), x_layer_db(translator, block_id=blo), z_layer_db(translator, block_id=blo), x_layer_db(translator, block_id=blo)])
circuit_db = concatenate_dbs([block_0, block_1])
circuit, circuit_db = translator.give_circuit(circuit_db)

In [26]:
simplified_db, ns = simplifier.reduce_circuit(circuit_db)
ss, simplified_db = translator.give_circuit(simplified_db)

<bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 0x7fce1e41de10>>
<bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 0x7fce1e41de10>>
<bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 0x7fce1e41de10>>
<bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 0x7fce1e41de10>>
<bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 0x7fce1e41de10>>
<bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 0x7fce1e41de10>>


In [27]:
mdif = max_diff(translator , circuit_db, simplified_db)
mdif

7.728927626074843e-05

In [59]:
b=0
c1 = concatenate_dbs([z_layer_db(translator, block_id =b),z_layer_db(translator, block_id =b), z_layer_db(translator, block_id =b), z_layer_db(translator, block_id =b)])

b=1
c2 = concatenate_dbs([z_layer_db(translator, block_id =b),z_layer_db(translator, block_id =b), z_layer_db(translator, block_id =b), z_layer_db(translator, block_id =b)])

ccdb = concatenate_dbs([c1, c2 ])
circuit, circuit_db = translator.give_circuit(ccdb)



In [61]:
simplifier = Simplifier(translator)
simplifier.relative_rules = [simplifier.rule_5]
simplifier.absolute_rules = [simplifier.rule_5]
simplifier.reduce_circuit(circuit_db)

<bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 0x7fce1c29d9e8>>
<bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 0x7fce1c29d9e8>>
<bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 0x7fce1c29d9e8>>
<bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 0x7fce1c29d9e8>>
<bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 0x7fce1c29d9e8>>
<bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 0x7fce1c29d9e8>>


(    ind symbol  param_value  trainable  block_id
 0     6   th_0    -0.786269       True         0
 1     7   th_1     2.414939       True         0
 2     8   th_2     6.596147       True         0
 3     9   th_3     6.283450       True         0
 4    10   th_4    12.566362       True         0
 5    11   th_5    -6.283208       True         0
 6     6   th_6   -12.267027       True         0
 7     7   th_7     3.786393       True         0
 8     8   th_8    -6.212194       True         0
 9     6   th_9    14.572547       True         1
 10    7  th_10    12.663116       True         1
 11    8  th_11    11.802699       True         1
 12    9  th_12    12.566414       True         1
 13   10  th_13    -0.000034       True         1
 14   11  th_14    12.566362       True         1
 15    6  th_15     6.560679       True         1
 16    7  th_16     2.109740       True         1
 17    8  th_17   -31.507799       True         1,
 8)

In [ ]:
b=0
c1 = u2_layer(translator,block_id=0)

b=1
c2 = concatenate_dbs([z_layer_db(translator, block_id =b),z_layer_db(translator, block_id =b), z_layer_db(translator, block_id =b), z_layer_db(translator, block_id =b)])

ccdb = concatenate_dbs([c1, c2 ])
circuit, circuit_db = translator.give_circuit(ccdb)



In [23]:
block_0 = concatenate_dbs([z_layer_db(translator, block_id=0), x_layer_db(translator, block_id=0), z_layer_db(translator, block_id=0), x_layer_db(translator, block_id=0)])
block_1 = concatenate_dbs([z_layer_db(translator, block_id=blo), x_layer_db(translator, block_id=blo), z_layer_db(translator, block_id=blo), x_layer_db(translator, block_id=blo)])
circuit_db = concatenate_dbs([block_0, block_1])

circuit_db = concatenate_dbs([u2_layer(translator, block_id=0), circuit_db])
circuit_db = concatenate_dbs([circuit_db, u2_layer(translator, block_id=1)])
circuit, circuit_db = translator.give_circuit(circuit_db)

In [ ]:
ci

In [15]:
simplifier.relative_rules = [simplifier.rule_5]
simplifier.absolute_rules = [simplifier.rule_5]

In [16]:
simplified_db, ns = simplifier.reduce_circuit(circuit_db)
ss, simplified_db = translator.give_circuit(simplified_db)

<bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 0x7fcec00827f0>>
<bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 0x7fcec00827f0>>
<bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 0x7fcec00827f0>>
<bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 0x7fcec00827f0>>
<bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 0x7fcec00827f0>>
<bound method Simplifier.rule_5 of <utilities.simplifier.Simplifier object at 0x7fcec00827f0>>
warning, repeated symbol while constructing the circuit, see circuut_
  symbol_name th_101
 symbols ['th_57', 'th_58', 'th_59', 'th_60', 'th_61', 'th_62', 'th_63', 'th_64', 'th_65', 'th_67', 'th_68', 'th_84', 'th_85', 'th_86', 'th_87', 'th_88', 'th_89', 'th_90', 'th_91', 'th_92', 'th_93', 'th_94', 'th_95', 'th_96', 'th_97', 'th_98', 'th_99', 'th_100', 'th_101']
circuit_db {0: {'ind': 6, 'symbol': 'th_57', 'param_value': 3.9115

warning, repeated symbol while constructing the circuit, see circuut_
  symbol_name th_90
 symbols ['th_57', 'th_58', 'th_59', 'th_60', 'th_61', 'th_62', 'th_63', 'th_64', 'th_65', 'th_82', 'th_83', 'th_84', 'th_85', 'th_86', 'th_87', 'th_88', 'th_89', 'th_90']
circuit_db {0: {'ind': 6, 'symbol': 'th_57', 'param_value': 3.9115450382232666, 'trainable': True, 'block_id': 1}, 1: {'ind': 7, 'symbol': 'th_58', 'param_value': 11.212042808532715, 'trainable': True, 'block_id': 1}, 2: {'ind': 8, 'symbol': 'th_59', 'param_value': 29.755796432495117, 'trainable': True, 'block_id': 1}, 3: {'ind': 9, 'symbol': 'th_60', 'param_value': 8.512166023254395, 'trainable': True, 'block_id': 1}, 4: {'ind': 10, 'symbol': 'th_61', 'param_value': -78.89285278320312, 'trainable': True, 'block_id': 1}, 5: {'ind': 11, 'symbol': 'th_62', 'param_value': -23.993059158325195, 'trainable': True, 'block_id': 1}, 6: {'ind': 6, 'symbol': 'th_63', 'param_value': 47.85822296142578, 'trainable': True, 'block_id': 1}, 7: {

True

In [143]:
mdif = max_diff(translator , circuit_db, simplified_db)
mdif

warning, repeated symbol while constructing the circuit, see circuut_
  symbol_name th_36
 symbols ['th_0', 'th_1', 'th_2', 'th_3', 'th_4', 'th_5', 'th_6', 'th_7', 'th_8', 'th_9', 'th_10', 'th_11', 'th_12', 'th_13', 'th_14', 'th_15', 'th_16', 'th_17', 'th_18', 'th_19', 'th_20', 'th_21', 'th_22', 'th_23', 'th_24', 'th_25', 'th_26', 'th_27', 'th_28', 'th_29', 'th_30', 'th_31', 'th_32', 'th_33', 'th_34', 'th_35', 'th_36']
circuit_db {0: {'ind': 6, 'symbol': 'th_0', 'param_value': 11.832184790948853, 'trainable': True, 'block_id': 0}, 1: {'ind': 9, 'symbol': 'th_1', 'param_value': -8.468219930588372, 'trainable': True, 'block_id': 0}, 2: {'ind': 7, 'symbol': 'th_2', 'param_value': 6.090899153526673, 'trainable': True, 'block_id': 0}, 3: {'ind': 10, 'symbol': 'th_3', 'param_value': -7.370951742521775, 'trainable': True, 'block_id': 0}, 4: {'ind': 7, 'symbol': 'th_4', 'param_value': 12.212132076394603, 'trainable': True, 'block_id': 0}, 5: {'ind': 0, 'symbol': None, 'param_value': -2.5988447

1.1510704060108723